In [ ]:
import pandas as pd
import numpy as np
import itertools

def calculate_angle(p1, p2, p3):
    """Calculates the angle at p2 formed by p1-p2-p3."""
    v1 = p1 - p2
    v2 = p3 - p2
    dot_product = np.dot(v1, v2)
    norm_product = np.linalg.norm(v1) * np.linalg.norm(v2)

    # Avoid division by zero
    if norm_product == 0:
        return np.nan

    # Clip the cosine value to [-1, 1] to avoid floating point errors
    cosine_angle = np.clip(dot_product / norm_product, -1.0, 1.0)

    angle_rad = np.arccos(cosine_angle)
    return np.degrees(angle_rad)

# --- 1. Load data and define skeleton connections ---
try:
    df = pd.read_csv('keypoints.csv', header=None)
except FileNotFoundError:
    print("Error: 'keypoints.csv' not found. Please ensure the file is in the correct path.")
    # Create a dummy DataFrame to allow the code to run; replace with your actual data
    df = pd.DataFrame(np.random.rand(10, 34))

connections = [[0, 1], [1, 2], [2, 3], [0, 4], [4, 5],
               [5, 6], [0, 7], [7, 8], [8, 9], [9, 10],
               [8, 11], [11, 12], [12, 13], [8, 14], [14, 15], [15, 16]]

# --- 2. Automatically calculate joint angle features ---
print("--- Starting Postural Feature Calculation ---")
postural_features = {}

# Select a central keypoint, e.g., 8
central_kp = 8
neighbors = [conn[1] if conn[0] == central_kp else conn[0] for conn in connections if central_kp in conn]

print(f"Found neighbors connected to keypoint {central_kp}: {neighbors}")

# Calculate the angles between all pairs of neighbors
for p1_idx, p3_idx in itertools.combinations(neighbors, 2):
    angles_over_time = [
        calculate_angle(
            np.array([row[p1_idx * 2], row[p1_idx * 2 + 1]]),
            np.array([row[central_kp * 2], row[central_kp * 2 + 1]]),
            np.array([row[p3_idx * 2], row[p3_idx * 2 + 1]])
        )
        for index, row in df.iterrows()
    ]

    angle_name = f"angle_{p1_idx}-{central_kp}-{p3_idx}"
    postural_features[f'mean_{angle_name}'] = np.nanmean(angles_over_time)
    postural_features[f'std_{angle_name}'] = np.nanstd(angles_over_time)

# --- 3. Calculate body sway features ---
# Assumption: 5=L_Shoulder, 6=R_Shoulder, 11=L_Hip, 12=R_Hip
L_SHOULDER, R_SHOULDER, L_HIP, R_HIP = 5, 6, 11, 12
torso_center_x = df[[L_SHOULDER*2, R_SHOULDER*2, L_HIP*2, R_HIP*2]].mean(axis=1)
torso_center_y = df[[L_SHOULDER*2+1, R_SHOULDER*2+1, L_HIP*2+1, R_HIP*2+1]].mean(axis=1)

postural_features['body_sway_x'] = torso_center_x.std()
postural_features['body_sway_y'] = torso_center_y.std()

print("\nCalculated Postural Features:")
for name, value in postural_features.items():
    print(f"{name}: {value:.2f}")

In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq

# --- 1. Set parameters and load data ---
# Important: Please change the sampling rate according to your actual setup!
SAMPLING_RATE = 30  # Assuming 30 frames per second (Hz)
# The keypoint index you want to analyze (e.g., 10 for the right wrist)
KEYPOINT_TO_ANALYZE = 10
# The typical frequency range for Parkinsonian tremor (Hz)
TREMOR_BAND = [4, 6]

try:
    df = pd.read_csv('keypoints.csv', header=None)
except FileNotFoundError:
    print("Error: 'keypoints.csv' not found. Please ensure the file is in the correct path.")
    # Create a dummy DataFrame to allow the code to run; replace with your actual data
    df = pd.DataFrame(np.random.rand(10, 34))

# --- 2. Extract signal and perform FFT ---
print("\n--- Starting Frequency-Domain Feature Calculation ---")
frequency_features = {}

# We will analyze the x and y signals separately
for axis in ['x', 'y']:
    col_idx = KEYPOINT_TO_ANALYZE * 2 + (0 if axis == 'x' else 1)

    if col_idx >= df.shape[1]:
        print(f"Warning: Column index {col_idx} for keypoint {KEYPOINT_TO_ANALYZE}'s {axis}-coordinate is out of bounds.")
        continue

    # Extract the time-series signal
    signal = df[col_idx].values
    N = len(signal)

    if N == 0:
        continue

    # Calculate FFT
    yf = fft(signal)
    xf = fftfreq(N, 1 / SAMPLING_RATE)

    # We only analyze the positive frequencies
    xf_positive = xf[:N//2]
    yf_positive_amplitude = 2.0/N * np.abs(yf[0:N//2])

    # --- 3. Extract frequency-domain features ---
    # Find the dominant frequency (the frequency with the highest amplitude)
    dominant_freq_idx = np.argmax(yf_positive_amplitude)
    dominant_freq = xf_positive[dominant_freq_idx]
    dominant_freq_amplitude = yf_positive_amplitude[dominant_freq_idx]

    frequency_features[f'dominant_freq_{axis}'] = dominant_freq
    frequency_features[f'dominant_freq_amplitude_{axis}'] = dominant_freq_amplitude

    # Calculate the signal power within the tremor band
    tremor_band_indices = np.where((xf_positive >= TREMOR_BAND[0]) & (xf_positive <= TREMOR_BAND[1]))
    power_in_tremor_band = np.sum(yf_positive_amplitude[tremor_band_indices]**2)
    frequency_features[f'power_in_tremor_band_{axis}'] = power_in_tremor_band

print("\nCalculated Frequency-Domain Features (for keypoint " + str(KEYPOINT_TO_ANALYZE) + "):")
for name, value in frequency_features.items():
    print(f"{name}: {value:.2f}")